# Winery classification with the multivariate Gaussian

Here we perform winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. 

There are 178 data points, each with 13 features and a label (1,2,3). We will divide this into a training set of 130 points and a test set of 48 points.

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [2]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [3]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [23]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

## 3. Use the model to make predictions on the test set

Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It returns the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

prints 7/48.

In [15]:
mu.shape

(4, 13)

In [24]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    k = 3 # Labels 1,2,...,k
    nt = len(testy) # Number of test points
    score = np.zeros((nt,k+1))
    for i in range(0,nt):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i,features], mean=mu[label,features], cov=sigma[label,features,features])
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    # Finally, tally up score
    errors = np.sum(predictions != testy)
    print( "Test error using feature(s): ")
    for f in features:
        print( "'" + featurenames[f] + "'" + " ")
    print( "Errors: " + str(errors) + "/" + str(nt))# Now test the performance of a predictor based on a subset of features    


### <font color="magenta">Fast exercises</font>

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [17]:
test_model(mu, sigma, pi, [2], testx, testy)

Test error using feature(s): 
'Ash' 
Errors: 29/48


Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [25]:
test_model(mu, sigma, pi, [0,2], testx, testy)

Test error using feature(s): 
'Alcohol' 
'Ash' 
Errors: 12/48


Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [26]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

Test error using feature(s): 
'Alcohol' 
'Ash' 
'Flavanoids' 
Errors: 4/48


Exercise 4. How many errors when using all 13 features?

In [20]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

Test error using feature(s): 
'Alcohol' 
'Malic acid' 
'Ash' 
'Alcalinity of ash' 
'Magnesium' 
'Total phenols' 
'Flavanoids' 
'Nonflavanoid phenols' 
'Proanthocyanins' 
'Color intensity' 
'Hue' 
'OD280/OD315 of diluted wines' 
'Proline' 
Errors: 2/48
